In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [4]:
dataset_january = pd.read_parquet('.data/.fhv_tripdata_2021-01.parquet')
dataset_february = pd.read_parquet('.data/.fhv_tripdata_2021-02.parquet')

Q1. Read the data for January. How many records are there? -> 1154112

In [5]:
dataset_january.shape

(1154112, 7)

Q2. What's the average trip duration in January? -> 19 min 10 s 

In [6]:
dataset_january['trip_duration'] = dataset_january['dropOff_datetime'] - dataset_january['pickup_datetime'] 

In [7]:
dataset_january.trip_duration.mean()

Timedelta('0 days 00:19:10.033445627')

Little EDA

In [8]:
#dataset_january.info()

In [9]:
# sns.kdeplot(dataset_january.pickup_datetime)
# plt.show()

In [10]:
# sns.kdeplot(dataset_january.pickup_datetime)

# plt.show()

In [11]:
# Convert trip_duration from date to the total number of minutes
dataset_january['trip_duration'] = dataset_january['trip_duration'].dt.total_seconds() / 60



In [12]:
n_before = dataset_january.shape[0]
#Keep only records with trip_duration between 1 and 60 minutes
dataset_january = dataset_january[(dataset_january.trip_duration > 1) & (dataset_january.trip_duration < 60)]

In [13]:
n_after = dataset_january.shape[0]

In [14]:
res = n_before - n_after
print(res)

46601


In [15]:
print(dataset_january.trip_duration.max())
print(dataset_january.trip_duration.min())

59.983333333333334
1.0000000000000002


Q3. The features we'll user for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1"

What's the factions of missing values for the pickup location ID? (Or the fraction of "-1"s after you filled the NAs)

In [16]:
dataset_january = dataset_january[['PUlocationID', 'DOlocationID', 'trip_duration']]
dataset_january = dataset_january.fillna(-1)

In [17]:

print((dataset_january.shape[0] - dataset_january[dataset_january.PUlocationID == -1].shape[0]) / dataset_january.shape[0])

0.16383403866869042


Q4. Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model :

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

 What's the dimensionality of this matrix? (The number of columns)

In [18]:
y = dataset_january.trip_duration
X = dataset_january.drop(columns=['trip_duration'])

In [19]:
# Convert dataset_january into a list of dictionaries
X = X.to_dict('records')
# Fit a dictionary vectorizer
vectorizer = DictVectorizer(sparse = False)
X = vectorizer.fit_transform(X)

one_hot_encoder = OneHotEncoder(sparse=False)
X = one_hot_encoder.fit_transform(X)



In [20]:
X.shape[1]

525

Q5. Now let's use the feature matrix from the previous step to train a model :

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?



Q6. Now let's apply this model to the validation dataset.

What's the RMSE on validation?



In [21]:
# Create train test split using X
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Train and evaluate a linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print(mean_squared_error(y_test, y_pred, squared = False))




10.395031820590392


In [22]:
preds_train = regressor.predict(X_train)
print(mean_squared_error(y_train, preds_train, squared = False))

10.393601814951502
